In [1]:
import seaborn as sns

import sklearn
from sklearn.cluster.bicluster import SpectralBiclustering
from sklearn.cluster.bicluster import SpectralCoclustering
from sklearn.metrics import consensus_score

import torch as T
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import scipy.linalg
from functools import partial
%pylab
%matplotlib inline
import matplotlib as mpl
import matplotlib.gridspec as gridspec
import scipy.sparse as sparse
from scipy import stats
from __future__ import print_function
import gc
from tqdm import tqdm_notebook as tqdm
from __future__ import division
import pandas as pd
import seaborn as sb
import os
from scipy.spatial import distance
from scipy.cluster import hierarchy
from pandas import DataFrame

import os, sys, datetime
LF_CODE_PATH = os.path.expanduser('~/projects/LFAnalyze/code')
FT_CODE_PATH = os.path.expanduser('~/projects/fishTrax/code/analysis/')
FD_CODE_PATH = os.path.expanduser('~/projects/fish_despair_notebooks/src/')
sys.path.insert(0,LF_CODE_PATH)
sys.path.insert(0,FT_CODE_PATH)
sys.path.insert(0,FD_CODE_PATH)

import passivity_2p_imaging_utils as p2putils
import visualization_utils as vizutil
reload(p2putils)
tmp_dir = '/tmp/'
all_data = p2putils.get_all_datasets(tmp_dir=tmp_dir)

f = all_data['e'][2]

# df, sig = f.get_cnmf_roi_table_and_signals()
M = f.get_signals_raw(z=None)

# data = np.load("../cnmf_f01555.npz")
# cnmf = data['cnmf'].astype(np.float32)
# raw = data['raw'].astype(np.float32)
# del data

def moving_average(a, n=3):
    # a is a signal
    ret = np.cumsum(a,0) # sum over time
    ret[n:] = ret[n:] - ret[:-n] # diff of n samples back
    rm = ret[n - 1:] / n
    pad_start = np.full((n-1,rm.shape[1]), rm[0])
    return np.vstack([pad_start, rm])

def ewma(data,span):
    "exponential weighted moving average."
    df = DataFrame(data)
    return df.ewm(span).mean().values

def df_f(x,ma_window=6,span=6):
    u = moving_average(x,ma_window)
    return ewma((x - u)/u, span)

# neurons = sig.T
neurons = M.T

neuron_ids = np.sort(np.argsort(neurons.std(0))[-5000:])
df = f.get_roi_table()
df = df.iloc[neuron_ids]
df.reset_index(drop=True, inplace=True)
neurons = neurons[:,neuron_ids]

neurons = T.from_numpy(df_f(neurons).astype(np.float32)).cuda()
neurons = (neurons - neurons.mean(0))/(neurons.std(0)+1e-8)

/home/ubuntu/anaconda3/envs/pytorch_p27/lib/python2.7/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/pytorch_p27/lib/python2.7/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
/home/ubuntu/anaconda3/envs/pytorch_p27/lib/python2.7/site-packages/scipy/special/__init__.py:640: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._ufuncs import *
/home/ubuntu/anaconda3/envs/pytorch_p27/lib/python2.7/site-packages/scipy/linalg/basic.py:17: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._solve_toeplitz import levinson
/home/ubuntu/anaconda3/envs/pytorch_p27/lib/python2.7/site-packages/scipy/linalg/__init__.py:207: RuntimeWarning: numpy.dtype size chang

Using matplotlib backend: agg
Populating the interactive namespace from numpy and matplotlib


/home/ubuntu/anaconda3/envs/pytorch_p27/lib/python2.7/site-packages/scipy/ndimage/measurements.py:36: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import _ni_label
/home/ubuntu/anaconda3/envs/pytorch_p27/lib/python2.7/site-packages/h5py/__init__.py:36: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._conv import register_converters as _register_converters
/home/ubuntu/anaconda3/envs/pytorch_p27/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/ubuntu/anaconda3/envs/pytorch_p27/lib/python2.7/site-packages/h5py/__init__.py:45: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88

In [ ]:
exclude_before = 1
exclude_after = 4
motion_frames = numpy.searchsorted(f.frame_et[:,-1], f.tail_movement_start_times,side="left")
exclude_idx = np.unique((motion_frames[:,None] + np.arange(-exclude_before,exclude_after+1)[None]).flatten())

In [ ]:
X = []
Y = []
diff = 5
for i, (x, y) in enumerate(zip(neurons[:-diff,None], neurons[diff:,None])):
    if i in set(list(exclude_idx)):
        continue
    else:
        X.append(x)
        Y.append(y)
X = np.vstack(X)
Y = np.vstack(Y)

In [ ]:
def corrcoef(x):
    # calculate covariance matrix of rows
    mean_x = torch.mean(x, 1, keepdim=True)
    xm = x.sub(mean_x.expand_as(x))
    c = xm.mm(xm.t())
    c = c / (x.size(1) - 1)

    # normalize covariance matrix
    d = torch.diag(c)
    stddev = torch.pow(d, 0.5)
    c = c.div(stddev.expand_as(c)+1e-8)
    c = c.div(stddev.expand_as(c).t()+1e-8)

    # clamp between -1 and 1
    # probably not necessary but numpy does it
    c = torch.clamp(c, -1.0, 1.0)

    return c

def idx_permute(start, theEnd, exclude_idx):
    num_before_start = np.sum(exclude_idx<start)
    num_before_end = np.sum(exclude_idx<theEnd)
    return start - num_before_start, theEnd - num_before_end

# idx_permute(1,10,np.arange(0,0))

class Granger(nn.Module):
    def __init__(self, nfeatures, n_past_steps, std=0.1, dtype=T.float32,bias=True):
        # we will perform nfeatures x nfeatures regressions
        super(Granger, self).__init__()
        if dtype==T.float32:
            tensor = T.cuda.FloatTensor
        elif dtype==T.float16:
            tensor = T.cuda.HalfTensor
        # Granger coefficient
        self.G = nn.Parameter(tensor(n_past_steps,nfeatures,nfeatures).normal_(std),
                              requires_grad=True)
        self.auto = nn.Parameter(tensor(n_past_steps,nfeatures,nfeatures).normal_(std),
                              requires_grad=True)
        self.bias = bias
        # Bias matrix
        if self.bias:
            self.B = nn.Parameter(tensor(nfeatures, nfeatures).normal_(std),requires_grad=True)

        self.n_past_steps = n_past_steps
        self.tensor = tensor
        self.dtype = dtype

    def forward(self, x_true, n_past_steps=None):
        if n_past_steps==None:
            n_past_steps = self.n_past_steps
        # nfeatures is n neurons & n_past_steps is lag
        # adding dim to make broadcast work
        # x_true is batch x lag x nfeatures x         1
        # G is          1 x lag x nfeatures x nfeatures
        y = self.G[None] * x_true[:,:,:,None]+ self.auto[None] * x_true[:,:,None,:]
        # sum lag so y is batch x nfeatures x nfeatures
        if self.bias:
            return y.sum(1) + self.B[None]
        else:
            return y.sum(1)
        
class Null(nn.Module):
    def __init__(self, nfeatures, n_past_steps, std=0.1, dtype=T.float32,bias=True):
        # we will perform nfeatures x nfeatures regressions
        super(Null, self).__init__()
        if dtype==T.float32:
            tensor = T.cuda.FloatTensor
        elif dtype==T.float16:
            tensor = T.cuda.HalfTensor
        self.auto = nn.Parameter(tensor(n_past_steps,nfeatures,nfeatures).normal_(std),
                              requires_grad=True)
        self.bias = bias
        # Bias matrix
        if self.bias:
            self.B = nn.Parameter(tensor(nfeatures, nfeatures).normal_(std),requires_grad=True)

        self.n_past_steps = n_past_steps
        self.tensor = tensor
        self.dtype = dtype

    def forward(self, x_true, n_past_steps=None):
        if n_past_steps==None:
            n_past_steps = self.n_past_steps
        # nfeatures is n neurons & n_past_steps is lag
        # adding dim to make broadcast work
        # x_true is batch x lag x nfeatures x         1
        y = self.auto[None] * x_true[:,:,None,:]
        # sum lag so y is batch x nfeatures x nfeatures
        if self.bias:
            return y.sum(1) + self.B[None]
        else:
            return y.sum(1)

def train(model,data,nepochs=1, lr=1e-3,batch_size=16, verbose=True):
    # l1 per parameter
    optimizer = torch.optim.Adam(model.parameters(), lr = lr)
    dataloader = DataLoader(data, batch_size=batch_size, shuffle=True)
    if verbose:
        epochs = range(nepochs)
    else:
        loader = dataloader
        epochs = tqdm(range(nepochs))
    for epoch in epochs:
        if verbose:
            loader = tqdm(dataloader)
        for X,Y in loader:
            Y_pred = model.forward(X)
            loss = F.mse_loss(Y[:,None,:], Y_pred)
            optimizer.zero_grad()
            loss.backward()# back props
            optimizer.step()# update the parameters
        if verbose:
            print('epoch {}, loss {}'.format(epoch,loss.data))
def residuals(model, data, batch_size=16, verbose=True):
    dataloader = DataLoader(data, batch_size=batch_size)
    loss = False
    if verbose:
        loader = tqdm(dataloader)
    else:
        loader = dataloader
    for X,Y in loader:
        with T.no_grad():
            Y_pred = model(X)
            l = ((Y[:,None,:] - Y_pred)**2).sum(0)
        if type(loss)!=T.Tensor:
            loss = l
        loss += l
    return loss

def f_value(resA, resB, n_params_A, n_params_B, n):
    return ( (resA-resB)/(n_params_B-n_params_A) ) / (resB/(n-n_params_B))


# def granger(X, Y, lag=1, batch_size=16):
#     g_model = nn.DataParallel(Granger(X.shape[1],1,bias=False))
#     null_model = nn.DataParallel(Null(X.shape[1],1,bias=False))
#     data = T.utils.data.TensorDataset(T.from_numpy(X[:,None]).cuda(),
#                                   T.from_numpy(Y).cuda())
#     train(g_model,data,3,lr=1e-1,batch_size=batch_size,verbose=False)
#     train(g_model,data,15,lr=1e-3,batch_size=batch_size,verbose=False)
#     train(null_model,data,3,lr=1e-1,batch_size=batch_size,verbose=False)
#     train(null_model,data,15,lr=1e-3,batch_size=batch_size,verbose=False)
#     g_res = residuals(g_model, data)
#     null_res = residuals(null_model, data)
#     n_params_g = 1+2*g_model.n_past_steps
#     n_params_null = 1+null_model.n_past_steps
#     f_vals = f_value(g_res, null_res, n_params_g, n_params_null, neurons.shape[0])
#     return f_vals



def granger(X, Y, lag=1, batch_size=16):

    g_beta, _ = T.gesv(y,x)
    null_beta, _ = T.gesv(y,x)
    residuals = T.matmul(x,beta)[:,:,0].sum(1)
    g_res = residuals(g_model, data)
    null_res = residuals(null_model, data)
    n_params_g = 1+2*g_model.n_past_steps
    n_params_null = 1+null_model.n_past_steps
    f_vals = f_value(g_res, null_res, n_params_g, n_params_null, neurons.shape[0])
    return f_vals


# data = T.utils.data.TensorDataset(T.from_numpy(X[:,None]).cuda(),
#                               T.from_numpy(Y).cuda())
# g_model = Granger(X.shape[1],1,bias=False)

In [ ]:
T.cat([T.ones(3,3),T.ones(3,3)],1).shape

In [ ]:
1+True

In [ ]:
def construct_pairwise_batches(X, lag=1, bias=True):
    "i*ncol+j index is granger model of influence from i->j."
    nT, N = X.shape
    granger = T.ones([N**2,nT-lag,2*lag+bias])
    null = T.ones([N**2,nT-lag,lag+bias])
    newY = T.ones([N**2,nT-lag,1])
    Y = X[lag:]
    # time x neuron x lag
    X_lag = T.cat([X[l:-(lag-l)][:,:,None] for l in reversed(range(lag))], 2)
    biasTerm = T.ones(nT-lag,1)
    for n1 in range(N):
        for n2 in range(N):
            if bias:
                grow = T.cat([X_lag[:,n1], X_lag[:,n2], biasTerm], 1)
                nrow = T.cat([X_lag[:,n2], biasTerm], 1)
            else:
                grow = T.cat([X_lag[:,n1], X_lag[:,n2]], 1)
                nrow = T.cat([X_lag[:,n2]], 1)
            granger[n1*N+n2] = grow
            null[n1*N+n2] = nrow
            newY[n1*N+n2] = Y[:,[n2]]
    return newY, granger, null

def batch_inverse(b_mat):
    eye = b_mat.new_ones(b_mat.size(-1)).diag().expand_as(b_mat)
    b_inv, _ = torch.gesv(eye, b_mat)
    return b_inv

def ols(Y, X):
    X_t = X.permute(0,2,1)
    sq = T.matmul(X_t,X)
    return T.matmul(T.matmul(batch_inverse(sq,
                     X_t),
                Y)

x = T.rand(7,3)
Y, x_granger, x_null = construct_pairwise_batches(x,3, True)

gbeta = ols(y,x_granger)
nbeta = ols(y,x_null)
g_residuals = T.matmul(x_granger,gbeta)[:,:,0].sum(1)
n_residuals = T.matmul(x_null,nbeta)[:,:,0].sum(1)

In [ ]:
def lm(X, Y, bias=True):
    "Fast linear regression"
    Y = Y[:,None]
    if bias:
        X = T.cat((X,T.ones(X.shape[0],1)),1)
    res = T.mm(T.inverse(T.mm(X.t(),X)), X.t())
    beta = T.mm(res, Y)
    residuals = ((Y-T.mm(X,beta))**2).sum()
    return beta, residuals

x = T.rand(20,5)
y = T.rand(20)
A = np.hstack([x.numpy(), np.ones([20,1])])
beta, residuals = lm(x, y)
np_beta, np_res = np.linalg.lstsq(A, y, rcond=None)[0:2]
print(beta[:,0].numpy(),"\n", np_beta)
print(float(residuals), np_res)

In [ ]:
# multiple models
def lm_mult(X, Y, bias=True):
    "Fast linear regression"
    Y = Y[:,None]
    if bias:
        X = T.cat((X,T.ones(X.shape[0],1)),1)
    res = T.matmul(T.inverse(T.matmul(X.t(),X)), X.t())
    beta = T.matmul(res, Y)
    residuals = ((Y-T.matmul(X,beta))**2).sum()
    return beta, residuals

x = T.rand(20,5)
y = T.rand(20)
A = np.hstack([x.numpy(), np.ones([20,1])])
# beta, residuals = lm(x, y)
np_beta, np_res = np.linalg.lstsq(A, y, rcond=None)[0:2]
print(beta[:,0].numpy(),"\n", np_beta)
print(float(residuals), np_res)

In [ ]:
T.version.cuda

In [ ]:
np_beta.shape

In [ ]:
x = T.rand(2,10,10)
y = T.rand(2,10,1)
beta, _ = T.gesv(y,x)
residuals = T.matmul(x,beta)[:,:,0].sum(1)
print(residuals.shape)
np_beta, np_res = np.linalg.lstsq(x[0], y[0], rcond=None)[0:2]
print(beta[0,:,0].numpy(),"\n", np_beta[:,0])
print(float(residuals[0]), np_res)

In [ ]:
np_res

In [ ]:
T.inverse(x)

In [ ]:
x = T.rand(2800,2)
y = T.rand(2800)
A = np.hstack([x.numpy(), np.ones([len(x),1])])

In [ ]:
5000**2/1000/60

In [ ]:
%%timeit
np_beta, np_res = np.linalg.lstsq(A, y, rcond=None)[0:2]

In [ ]:
y.shape

In [ ]:
(T.mm(T.from_numpy(A.astype(np.float32)),beta)).shape

In [ ]:
beta

In [ ]:
np_beta

In [ ]:
residuals

In [ ]:
np_res

In [ ]:
plt.matshow(A)

In [ ]:
A.shape

In [ ]:
x = T.rand(10,10)
plt.matshow(torch.mm(torch.inverse(x), x))

In [ ]:
G = granger(X,Y)

In [ ]:
plt.hist(G.cpu().numpy()[:])

In [ ]:
plt.imshow(G)

In [ ]:
import gc

gc.collect()

T.cuda.empty_cache()

In [ ]:
def cm_cycle(i):
    i = i - 1
    cmap = plt.get_cmap("tab10")
    color = cmap(i%10)
    if i%10==7:
        color = (color[0],color[1],color[2]+0.4)
    return color

In [ ]:
row_corr = corrcoef(G).cpu().numpy()

row_linkage = hierarchy.linkage(
    distance.pdist(row_corr), method='ward')

In [ ]:
col_corr = corrcoef(G.t().detach()).cpu().numpy()
col_linkage = hierarchy.linkage(
    distance.pdist(col_corr.T), method='ward')

In [ ]:
g_nclust = 10
row_clusters = hierarchy.fcluster(row_linkage,g_nclust,criterion='maxclust')
col_clusters = hierarchy.fcluster(col_linkage,g_nclust,criterion='maxclust')

In [ ]:
vmax = max(np.abs(np.percentile(G,0.01)), np.abs(np.percentile(G,0.99)))

In [ ]:
cm = sb.clustermap(G.cpu().numpy(), row_linkage=row_linkage, row_colors=[cm_cycle(c) for c in row_clusters],
        col_linkage=col_linkage, col_colors=[cm_cycle(c) for c in col_clusters],
        figsize=(20, 20),cmap="RdBu_r", vmin=-vmax,vmax=vmax)

In [ ]:
Z = [0,2,4,6,8,10]
# Z = [4]
nZ = len(Z)
back_img = []
for z in Z:
    back_img.append(np.power(f.get_tif_rasl_as_vol(z,range(1,200)).mean(axis=2),.4))
    

In [ ]:
# nrows = len(row_clust_to_plot)
# nZ = 1
nrows = g_nclust
plt.subplots(nrows,nZ, figsize=[8*nZ,4*nrows])
for ic, clust in enumerate(range(1, g_nclust+1)):
# for ic,clust in enumerate(row_clust_to_plot):
    for iz, z in enumerate(Z):
        #Select rois in raphe in this slices, and get their coordinates.
#         coords = df[(row_clusters==clust)].coords
        coords = df[(row_clusters==clust) & (df.z==z)].coords
#         poly_coords = df[(row_clusters==clust) & (df.z==z)].poly
#         poly_coords = df[(row_clusters==clust)].poly
#         coords = [np.round(poly[~np.isnan(poly).any(axis=1)])[:,(1,0)].astype(int) for poly in poly_coords]
        plt.subplot(nrows,nZ,ic*nZ+iz+1)
        #Overlay the ROIs on the background image and display:
        # hack iz hardcode
        img = vizutil.overlay_coords(back_img[iz], coords, list(cm_cycle(clust)[:3]), alpha=1)
        plt.imshow(img,interpolation='nearest')
        plt.title("Row cluster {}, z={}".format(clust,z),fontsize=18)

In [ ]:
# nrows = len(col_clust_to_plot)
plt.subplots(nrows,nZ, figsize=[8*nZ,4*nrows])
for ic,clust in enumerate(range(1,g_nclust+1)):
# for ic,clust in enumerate(col_clust_to_plot):
    for iz, z in enumerate(Z):
        #Select rois in raphe in this slices, and get their coordinates.
        coords = df[(col_clusters==clust) & (df.z==z)].coords
#         poly_coords = df[(row_clusters==clust) & (df.z==z)].poly
#         coords = df[(col_clusters==clust)].coords
#         poly_coords = df[(col_clusters==clust)].poly
#         coords = [np.round(poly[~np.isnan(poly).any(axis=1)])[:,(1,0)].astype(int) for poly in poly_coords]
        plt.subplot(nrows,nZ,ic*nZ+iz+1)
        #Overlay the ROIs on the background image and display:
        img = vizutil.overlay_coords(back_img[iz], coords, list(cm_cycle(clust)[:3]), alpha=1)
        plt.imshow(img,interpolation='nearest')
        plt.title("Col cluster {}, z={}".format(clust,z),fontsize=18)

In [ ]:
s = 0
e = 440
fig, ax = plt.subplots(1,2, figsize=(5,100))
ax[0].imshow(X[slice(*idx_permute(s,e,exclude_idx))].T)
ax[1].imshow(x_fish[s:e].T)

In [ ]:
colidx = col_clusters==34
rowidx = row_clusters==7
plt.matshow(G[rowidx][:,colidx],vmin=-0.1,vmax=0.1)
plt.colorbar()

In [ ]:
ind = slice(*idx_permute(2400,2800, exclude_idx))
# ind = slice(*idx_permute(0,400, exclude_idx))

In [ ]:
# model = Granger(len(neuron_ids),1,bias=False)
model = Granger(X.shape[1],1,bias=False)
G = model.G.data[0].detach().cpu().numpy()
# trunc_fish = x_fish[0:400]
# trunc_fish = x_fish[440:840]
# trunc_fish = x_fish[840:1240]
# trunc_fish = x_fish[1240:1640]
# trunc_fish = x_fish[1640:2040]
# trunc_fish = x_fish[1640:2440]
# trunc_fish = x_fish[640:840]
# trunc_fish = x_fish[2400:2800]
# trunc_fish = x_fish[2600:2800]
# trunc_fish = x_fish
# data = T.utils.data.TensorDataset(T.from_numpy(X[ind,None,neuron_ids]).cuda(),
#                                   T.from_numpy(Y[ind,neuron_ids]).cuda())
data = T.utils.data.TensorDataset(T.from_numpy(X[ind,None]).cuda(),
                                  T.from_numpy(Y[ind]).cuda())
l1 = 5e-6
# train(model,data,10,l1=l1,batch_size=4) # max(bz)==4 for all neurons
train(model,data,3,l1p=1e-0,lr=1e-1,batch_size=1,verbose=False)
train(model,data,15,l1p=5e-2,lr=1e-3,batch_size=1,verbose=False)
G = model.G.data[0].detach().cpu().numpy()